TF

In [1]:
import seismic_ml_kernel

In [2]:
#train (train_data_dir, validation_data_dir, image_size)
#test (validation_data_dir + 'noise/', image_size)

In [3]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

#mn (X_train, y_train, X_test, y_test)

Using TensorFlow backend.


In [1]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

#ci (X_train, y_train, X_test, y_test)

Using TensorFlow backend.


In [ ]:
import seismic_dataset_builder
    
#noisers = [None, seismic_handler.FreqNoiser()]
noisers = [None,                                                # original gather
           seismic_dataset_builder.BP(0.1, 0.15, 0.2, 0.25, 1), # BP1
           seismic_dataset_builder.BP(0.2, 0.25, 0.3, 0.35, 1), # BP2
           seismic_dataset_builder.BP(0.3, 0.35, 0.4, 0.45, 1), # BP3
           seismic_dataset_builder.BP(0.4, 0.45, 0.5, 0.55, 1), # BP4
          ]
#noisers = [None, seismic_handler.BP(0.1, 0.15, 0.2, 0.25, 1)]

(X_train, y_train), (X_test, y_test) = seismic_dataset_builder.create_seis_dataset ( file_name ='data/prestack.sgy', 
                                                                                     sorting_key = 'original_field_record_number', 
                                                                                     window = [50, 0.5], 
                                                                                     noisers = noisers,
                                                                                     images_dir = None # dont create images
                                                                                    )

import seismic_ml_kernel
#seismic_ml_kernel.mn (X_train[0], y_train, X_test[0], y_test)


classification_model = seismic_ml_kernel.ci_multi_train (X_train, y_train)
seismic_ml_kernel.ci_multi_test (classification_model, X_test, y_test)

Using TensorFlow backend.


image has 4600 pixels


C:\Users\kostyad\Documents\Jupyter\seismic_fingerprint\seismic_ml_kernel.py:138: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("fl..., inputs=Tensor("in...)`
  vision_model = Model(input=inp, output=flat) # To define a model, just specify its input and output layers


image has 2300 pixels
image has 2350 pixels
number of vision models 3
Vision models are ready
Classification model is ready
Classification model compiled
Train on 12226 samples, validate on 1359 samples
Epoch 1/20
 5728/12226 [=============>................] - ETA: 461s - loss: 1.5954 - acc: 0.2301

# vision model 1
![title](outputs/vision_model0.png)

# vision model 2
![title](outputs/vision_model1.png)

# vision model 3
![title](outputs/vision_model2.png)

# classification model
![title](outputs/classification_model.png)

In [1]:
python -m tensorflow.tensorboard --logdir=outputs/Graph

SyntaxError: invalid syntax (<ipython-input-1-70e8466a5ff2>, line 1)